## Smishing detection using [danielhenel/smishing-detection-llama-2-7b-chat](https://huggingface.co/danielhenel/smishing-detection-llama-2-7b-chat) - evaluation of the model

In [1]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM
from transformers import BitsAndBytesConfig
import pickle
from transformers import pipeline

In [2]:
# test data
with open("./data/test_data.pkl", "rb") as input_file:
    test_data = pickle.load(input_file)

X_test = test_data["X_test"]
y_test = test_data["y_test"]

total_hams_count = 0
total_smishes_count = 0

for label in y_test:
    if label == "ham":
        total_hams_count += 1
    if label == "smish":
        total_smishes_count += 1

print("There is {} hams and {} smishes in the test dataset.".format(total_hams_count, total_smishes_count))

There is 954 hams and 161 smishes in the test dataset.


In [3]:
# Model name
base_model = 'NousResearch/Llama-2-7b-chat-hf'
adapter_model = 'danielhenel/smishing-detection-llama-2-7b-chat'

# Quantization configuration
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Load the model with quantization configuration
model = LlamaForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.load_adapter(adapter_model)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(adapter_model, trust_remote_code=True)

c:\Users\Daniel\Desktop\repo\research\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.86s/it]


In [4]:
import logging
logging.basicConfig(filename='fine_tuned_llama_2_7b.log', level=logging.INFO)

false_hams_indicies = []
false_smishes_indicies = []
false_hams_count = 0
false_smishes_count = 0
true_hams_count = 0
true_smishes_count = 0
errors_count = 0
errors_indicies = []
errors = []


for i in range(len(X_test)):
    prompt = (
    "<s>[INST] Do you think it is a ham or smish message? "
    "Your output should be a single word 'smish' or 'ham'. "
    "Do not write a sentence. "
    "Output is case-sensitive. "
    "SMS content: {}[/INST]"
    ).format(X_test[i])

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
    answer = pipe(prompt)
    answer = answer[0]['generated_text'].split("[/INST]")[1].split("</s>")[0].lower().strip()
    logging.info('{}::{}'.format(i,answer))

    if answer not in ["ham", "smish"]:
        errors_count += 1
        errors_indicies.append(i)
        errors.append(answer)
        continue
    elif answer == "ham" and y_test[i] == "ham": # correctly recognized as a ham
        true_hams_count += 1
    elif answer == "smish" and y_test[i] == "smish": # correctly recognized as a smish
        true_smishes_count += 1
    elif answer == "ham" and y_test[i] == "smish": # wrongly recognized as a ham
        false_hams_indicies.append(i)
        false_hams_count += 1
    elif answer == "smish" and y_test[i] == "ham": # wrongly recognized as a smish
        false_smishes_indicies.append(i)
        false_smishes_count += 1

# errors warning
if errors_count != 0:
    if errors_count == 1:
        print("WARNING: {} error".format(errors_count))
    else:
        print("WARNING: {} errors".format(errors_count))

# save results for further analysis
results = {"FN" : false_hams_count, "FP" : false_smishes_count,
           "TN" : true_hams_count, "TP" : true_smishes_count,
           "FN_indicies" : false_hams_indicies, "FP_indicies" : false_smishes_indicies,
            "errors_count" : errors_count, "errors" : errors, "errors_indicies" : errors_indicies}

with open("./results/results_llama_2_7b_fine_tuned.pkl", 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\Daniel\Desktop\repo\research\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### The performance of our fine-tuned Llama 2 7b model in smishing detection.

In [5]:
FN = false_hams_count     #FN - messages wrongly recognized as not smishes (hams)
FP = false_smishes_count  #FP - messages wrongly recognized as smishes
TN = true_hams_count      #TN - messages correctly recognized as not smishes (hams)
TP = true_smishes_count   #TP - messages correctly recognized as smishes
TOTAL = FN + FP + TN + TP

In [6]:
print("Messages wrongly recognized as hams: {0:.2f}%".format(FN / TOTAL * 100))
print("Messages wrongly recognized as smishes: {0:.2f}%".format(FP / TOTAL * 100))
print("Messages correctly recognized as hams: {0:.2f}%".format(TN / TOTAL * 100))
print("Messages correctly recognized as smishes: {0:.2f}%".format(TP / TOTAL * 100))

Messages wrongly recognized as hams: 0.27%
Messages wrongly recognized as smishes: 0.63%
Messages correctly recognized as hams: 84.93%
Messages correctly recognized as smishes: 14.17%


#### Accuracy

In [7]:
accuracy = (TP + TN) / TOTAL
print("{0:.2f}%".format(accuracy * 100))

99.10%


#### Recall

In [8]:
recall = TP / (TP + FN)
print("{0:.2f}%".format(recall * 100))

98.14%


#### Precision

In [9]:
precision = TP / (TP + FP)
print("{0:.2f}%".format(precision * 100))

95.76%


#### F1 score

In [10]:
F1_score = TP / (TP + (FP + FN) / 2)
print("{0:.2f}%".format(F1_score * 100))

96.93%
